# Use examples of [premise](https://github.com/polca/premise)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to adapt the life cycle inventory database [ecoinvent](https://www.ecoinvent.org/) for prospective environmental impact assessment.

This library extracts useful information from IAM output files (e.g., REMIND, IMAGE, GCAM, TIAM-UCL) and aligns inventories in the ecoinvent database accordingly.

As of premise 2.3.x, the following sector updates are available:

* `update("biomass")`: creates regional biomass markets and splits purpose-grown vs. residual biomass
* `update("battery")`: scales battery pack mass and creates mobile/stationary battery markets
* `update("electricity")`: creates regional electricity markets and updates generation efficiencies
* `update("cement")`: regionalizes clinker/cement production and adjusts kiln efficiency/CCS
* `update("steel")`: regionalizes steel production routes and adjusts process efficiencies
* `update("cdr")`: adds carbon dioxide removal routes (e.g., DACCS, BECCS) where mapped
* `update("fuels")`: creates regional markets for liquid and gaseous fuels
* `update("heat")`: regionalizes heat/steam production and relinks to fuel markets
* `update("emissions")`: scales pollutant emissions using GAINS projections
* `update("metals")`: updates metal supply chains and material intensity factors
* `update("mining")`: updates mining waste treatment and regional mining markets
* `update("renewable")`: updates wind turbine inventories and deployment where mapped
* `update("final energy")`: updates final energy carrier mixes for downstream sectors
* `update("cars")`, `update("two_wheelers")`, `update("trucks")`, `update("buses")`, `update("trains")`, `update("ships")`: update transport inventories and fleet averages
* `update("external")`: applies user-defined scenario datapackages

Alternatively, `update()` performs all the sector updates above.

Integration of user-defined scenarios (via `update("external")`) is also possible; see the separate notebook on user-defined scenarios.

Additional documentation on the methodology is available [here](https://premise.readthedocs.io/en/latest/introduction.html).

There's also a **publication** about `premise` [here](https://www.sciencedirect.com/science/article/pii/S136403212200226X?via%3Dihub).

## Requirements

* **Python 3.10 or higher (see `pyproject.toml`)**
* a user license for ecoinvent v.3 (3.5-3.12 supported; consequential requires 3.8+)
* a **decryption key** to download encrypted IAM scenario files (optional if you provide local IAM files)


## What Premise Does (at a glance)

Premise takes IAM scenario outputs and applies them to an ecoinvent LCI database by creating or modifying datasets and markets.

**High-level flow:**

    Extract ecoinvent + inventories
            |
            v
    Load IAM data (or external scenario data)
            |
            v
    Apply sector updates (electricity, fuels, steel, etc.)
            |
            v
    Export to Brightway / SimaPro / OpenLCA / matrices / datapackage

**Choose your path:**

- **Quickstart**: build a scenario and run `ndb.update()` (all sectors).
- **Targeted updates**: run `ndb.update(["electricity", "steel"])` to limit changes.
- **External scenarios**: pass `external scenarios` and call `ndb.update("external")`.
- **Consequential**: use an ecoinvent consequential DB and `system_model="consequential"` (3.8+).

The rest of this notebook shows concrete examples for each path.


## Quickstart (minimal run)

This is the smallest end-to-end run: create a database, apply all sector updates, and export to Brightway.
Adjust `model`, `pathway`, `year`, and `source_db` to match your setup.


In [ ]:
from premise import NewDatabase
import bw2data

bw2data.projects.set_current("premise")

ndb = NewDatabase(
    scenarios=[{"model": "remind", "pathway": "SSP2-NDC", "year": 2030}],
    source_db="ecoinvent-3.10-cutoff",
    source_version="3.10",
    # key="xxxxxxxxxxxxxxxxxxxxxxxxx",  # optional: required to download encrypted IAM files
)

ndb.update()
ndb.write_db_to_brightway()


If you only want a subset of updates, call `ndb.update(["electricity", "steel"])` instead of `ndb.update()`.


## Conceptual glossary

- **Scenario**: one IAM model + pathway + year (and optional external scenarios).
- **Sector update**: a transformation applied to a subset of the database (e.g., electricity, fuels, steel).
- **Proxy dataset**: a copy of an existing dataset used to represent a technology in a specific IAM region.
- **Regional market**: a market dataset that aggregates suppliers for a region based on production volumes and mappings.
- **Relinking**: replacing inputs in consuming activities so they draw from the new regional markets or proxies.
- **External scenario**: a datapackage with additional scenario data and optional inventories applied via `update("external")`.
- **Scenario report**: an Excel summary of key IAM variables used in your run.
- **Change report**: an Excel summary of dataset modifications and validation flags.


## Common pitfalls

- **Wrong `source_db` name**: it must match the database name in your Brightway project.
- **Missing biosphere**: ensure `biosphere3` (or your custom biosphere name) is registered.
- **IAM file not found**: check `premise/data/iam_output_files` or pass `filepath` to your own files.
- **Model/pathway mismatch**: make sure `model` and `pathway` are supported (see `constants.yaml`) or add your own.
- **Consequential with unsupported version**: consequential mode only works with ecoinvent 3.8+.
- **External scenarios not applied**: call `ndb.update("external")` or include it in `ndb.update([...])`.


## Troubleshooting triage

If a run fails, check these in order:

1. **Project & biosphere**: is the Brightway project active and does it include `biosphere3` (or your custom biosphere name)?
2. **Source DB name**: does `source_db` match exactly the database name in the project?
3. **IAM file naming**: if using local IAM files, are they named `{model}_{pathway}.csv/.mif/.xlsx` and located in `filepath`?
4. **Model/pathway validity**: are `model` and `pathway` in `constants.yaml` or in your custom mapping?
5. **Consequential mode**: are you using ecoinvent 3.8+ and `system_model="consequential"`?
6. **External scenarios**: did you call `ndb.update("external")` or include `"external"` in your update list?

If it still fails, check the change report and logs for the last dataset processed.


# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, you need an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.org) v.3 database (3.5-3.12; consequential requires 3.8+) registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.


In [3]:
from premise import *
import bw2data

### List of available scenarios

Some IAM scenario files are bundled with the library. They are stored in `premise/data/iam_output_files` from the root directory.
Examples include `remind_SSP1-NPi.csv`, `image_SSP1-L.csv`, `tiam-ucl_SSP2-RCP19.csv`, and `gcam_SSP2-Base.csv`.

If a scenario file is missing locally but you provide a decryption key, `premise` will download the default IAM output file from Zenodo on first use.

Supported model names and pathway names are listed in `premise/iam_variables_mapping/constants.yaml` (see `SUPPORTED_MODELS` and `SUPPORTED_PATHWAYS`).


### Database creation from default scenarios

To create a scenario using REMIND's SSP1-NPi pathway, from ecoinvent 3.10 for the year 2050, one would execute the following. This leads to the extraction of the database, some cleanup as well as importing a few additional inventories.

First, activate the brightway2 project where you want to store the database.


In [ ]:
# activate the bw project
bw2data.projects.set_current("premise")
# list currently installed databases
bw2data.databases

In [ ]:
ndb = NewDatabase(
    scenarios=[
        {"model":"image", "pathway":"SSP1-L", "year":2050},
        {"model":"remind", "pathway":"SSP1-NPi", "year":2050},
    ],
    source_db="ecoinvent-3.10-cutoff", # name of the database in the BW2 project. Must be a string.
    source_version="3.10", # 3.5-3.12 supported; consequential requires 3.8+.
    # key="xxxxxxxxxxxxxxxxxxxxxxxxx", # optional: required to download encrypted IAM files
    keep_source_db_uncertainty=False, # False by default, set to True if you want to keep ecoinvent's uncertainty data
    keep_imports_uncertainty=True, # True by default, set to False if you want to drop uncertainty for imported inventories
    use_absolute_efficiency=False, # False by default, set to True if you want to use IAM absolute efficiencies
)


Here is a list of all arguments that can be passed to `NewDatabase()`:
    
    scenarios: List[dict], # list of scenarios to process
    source_version: str = "3.12", # version of ecoinvent database (3.5-3.12; consequential requires 3.8+)
    source_type: str = "brightway", # type of source database. Can be "brightway" or "ecospold".
    key: bytes | str = None, # decryption key
    source_db: str = None, # name of database if source_type is "brightway"
    source_file_path: str = None, # path to ecospold files if source_type is "ecospold"
    additional_inventories: List[dict] = None, # list of additional inventories to import
    system_model: str = "cutoff", # system model. Can be "cutoff" or "consequential".
    system_args: dict = None, # arguments for the "consequential" system model
    use_cached_inventories: bool = True, # use cached inventories
    use_cached_database: bool = True, # use cached database
    external_scenarios: list = None, # list of external scenarios to use
    quiet: bool = False, # suppress output
    keep_imports_uncertainty: bool = True, # keep uncertainty data of additional inventories
    keep_source_db_uncertainty: bool = False, # keep uncertainty data of source database
    gains_scenario: str = "CLE", # air pollution GAINS scenario
    use_absolute_efficiency: bool = False, # use IAM absolute efficiencies instead of efficiencies relative to 2020
    biosphere_name: str = "biosphere3", # name of biosphere database in brightway project if different from "biosphere3"
    generate_reports: bool = True, # generate scenario/change reports on export


The first time you create a premise database, *premise* will store a copy of the ecoinvent database and external inventories, to be able to skip that time-consuming step next time. If you wish to clear this cache (which is only encouraged if updating premise or if encountering issues with inventories), do:


In [ ]:
clear_cache() # clears both ecoinvent and additional inventories cache
clear_inventory_cache() # clears only additional inventories cache

If you do not want to integrate the IAM projections in the database, but only wish to have the additional inventories, you can stop here and export the database back to Brightway or other destinations, by using the `write_db_to` methods, like so:

In [ ]:
ndb.write_db_to_brightway()

However, if you wish first to proceed with the IAM integration, you need to use the `update()` method, like so for the electricity sector:

In [ ]:
ndb.update("electricity")

In [ ]:
ndb.write_db_to_brightway()

If you want to create multiple databases at once, just populate the `scenarios` list.

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-NDC", "year":2020},
                {"model":"remind", "pathway":"SSP2-NDC", "year":2030},
                {"model":"remind", "pathway":"SSP2-NDC", "year":2040},
                {"model":"remind", "pathway":"SSP2-NDC", "year":2050},
            ],
            source_db="ecoinvent 3.7 cutoff", # name of the database. Must be a string.
            source_version="3.7.1", # 3.5-3.12 supported (3.7.1 is normalized to 3.7)
            key='xxxxxxxxxxxxxxxxxxxxxxxxx'
)




When the database is loaded and the additional inventories imported, you can apply a transformation function.
For example here, we adjust the efficiency of the power plants to the two scenarios we have loaded.
We go more in details later.

In [ ]:
ndb.update("electricity")

Or you can proceed instead to doing all the sectoral transformations available, like so:

In [ ]:
ndb.update() # <- updates all sectors

And then, we register these two databases back into brightway2.

In [ ]:
ndb.write_db_to_brightway()

### Consequential

`premise` can read in the consequential version of ecoinvent (3.8+).
Based on the publication of Maes et al. 2023 (https://doi.org/10.1016/j.rser.2023.113830), `premise` builds marginal market mixes for markets that use IAM production volumes, while excluding constrained suppliers (see `premise/data/consequential/constrained_suppliers.yaml`).
Additionally, `premise` removes secondary steel from consequential steel markets.
Passing a series of arguments to `NewDatabase()` can influence the identification of marginal suppliers.


In [ ]:
from premise import *
from datapackage import Package
import brightway2 as bw
bw.projects.set_current("new4")

args = {"range time":2, "duration":0, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00}

ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-NDC", "year":2020},
                {"model":"remind", "pathway":"SSP2-NDC", "year":2030},
                {"model":"remind", "pathway":"SSP2-NDC", "year":2040},
                {"model":"remind", "pathway":"SSP2-NDC", "year":2050},
            ],
            source_db="ecoinvent 3.8 consequential", # must point to a consequential database
            source_version="3.8", # 3.8-3.12 supported for consequential mode
            key='xxxxxxxxxxxxxxxxxxxxxxxxx',
            system_model="consequential", # must specify "consequential"
            system_args=args # optional arguments
)


### Database creation from non-default scenarios

If you have some specific IAM scenarios (not included in `premise`) from which you would like to build a database, you can specify the directory for those.

**Important remark**: your scenario file should be named `{model}_{pathway}` and stored as a `.csv`, `.mif`, or `.xlsx` file in the folder you pass via `filepath`. The `model` must be one of the supported model names (or added to `constants.yaml` with a topology file).
If the file exists locally, you do not need a decryption key.


In [ ]:
from premise import *
import bw2data

bw2data.projects.set_current("new")

In [ ]:
ndb = NewDatabase(
    scenarios = [{"model":"remind", "pathway":"my_special_scenario", "year":2028,
                  "filepath":"/Users/romain/Documents"}],        
    source_db="ecoinvent 3.8 cutoff", # name of the database
    source_version="3.8", # version of ecoinvent
 )


### Adding inventories
Upon the database extraction, you can import some of your Brightway2-compatible inventories like so:

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-NDC", "year":2030},
            ],
            source_db="ecoinvent 3.7 cutoff", 
            source_version="3.7.1",
            key='xxxxxxxxxxxxxxxxxxxxxxxxx',
            additional_inventories= [
                {"filepath": r"filepath\to\excel_file.xlsx", "ecoinvent version": "3.7"},
                {"filepath": r"filepath\to\another_excel_file.xlsx", "ecoinvent version": "3.7"},
            ]
                 )


# Use case with ecospold2

The source database does not have to be from a brightway2 project.
It can be directly extracted from the set of ecospold2 files obtained from the [ecoinvent website](https://ecoinvent.org).

For this, one needs to specify the argument `source_type = "ecospold"` and `source_file_path`, which is the directory leading to the ecospold files.

For example, here we combine the use of a specific (non-default) IAM scenario file with the use of ecospold2 files as data source (ecoinvent 3.6 in this case).


In [ ]:
ndb = NewDatabase(
        scenarios = [
            {"model":"remind", "pathway":"my_special_scenario", "year":2028,
             "filepath":r"C:\filepath\to\your\scenario\folder"}
        ],        
        source_type="ecospold", # <--- this is NEW
        source_file_path=r"C:\filepath\to\your\ecosposld\folder\datasets", # <-- this is NEW
        source_version="3.6",
    )

# Transformation functions

These functions modify the extracted database:

* **update("biomass")**: create scenario- and region-specific markets for biomass used for power generation, distinguishing purpose-grown and residual biomass.

* **update("battery")**: scale battery pack mass and create mobile/stationary battery markets.

* **update("electricity")**: align regional electricity production mixes and update efficiencies for several electricity production technologies.

* **update("cement")**: adjust cement/clinker production efficiency and (where mapped) CCS integration.

* **update("steel")**: adjust process efficiency and regional steel market shares.

* **update("cdr")**: create region- and scenario-specific inventories for carbon dioxide removal (e.g., DACCS, BECCS).

* **update("fuels")**: create regional markets for liquid and gaseous fuels and relink consuming activities.

* **update("heat")**: create regionalized heat and steam production datasets and relink them to regional fuel markets.

* **update("emissions")**: adjust emissions of local air pollutants according to GAINS projections.

* **update("metals")**: update metal supply chains and material intensity factors.

* **update("mining")**: update mining waste treatment and regional mining markets.

* **update("renewable")**: update wind turbine inventories and deployment where mapped.

* **update("final energy")**: update final energy carrier mixes for downstream sectors.

* **update("cars")**: create updated inventories for fleet average passenger cars and relink consuming activities.

* **update("trucks")**: create updated inventories for fleet average lorry trucks and relink consuming activities.

* **update("two_wheelers")**: create updated inventories for fleet average two-wheelers and relink consuming activities.

* **update("buses")**: create updated inventories for fleet average buses and relink consuming activities.

* **update("trains")**: create updated inventories for fleet average trains and relink consuming activities.

* **update("ships")**: create updated inventories for fleet average ships and relink consuming activities.

A look at the documentation is advised.

These functions can be applied *separately*, *consecutively* or *altogether* (using instead **.update()** without arguments).

They will apply to all the scenario-specific databases listed in `scenarios`.


In [ ]:
from premise import *
import bw2data
bw2data.projects.set_current("some project")

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind","pathway":"SSP2-NDC","year":2020},
                {"model":"image", "pathway":"SSP2-M", "year":2034},
            ],
            key='xxxxxxxxxxxxxxxxxxxxxxxxx',
            source_db="ecoinvent 3.7 cutoff",
            source_version="3.7", 
)


In [ ]:
ndb.update()

In [ ]:
# write back to brightway project
ndb.write_db_to_brightway()

You can also give your datababases a custom name.

In [ ]:
ndb.write_db_to_brightway(name=["my_custom_name_1", "my_custom_name_2"])

# Export options

### As a Brightway2 database

Export the modified database to brightway2

In [ ]:
ndb.write_db_to_brightway()

### As a SimaPro CSV file

In [ ]:
ndb.write_db_to_simapro(filepath=r"C:/Users/sacchi_r/Downloads/exported_simapro_file")

### As a CSV file for OpenLCA

In [ ]:
ndb.write_db_to_olca(filepath=r"C:/Users/sacchi_r/Downloads/exported_olca_file")

### As a Superstructure database
A superstructure database is a database that can accommodate several scenarios, as described [here](https://github.com/dgdekoning/brightway-superstructure), to be then used in [Activity-Browser](https://github.com/LCA-ActivityBrowser/activity-browser).
This function will export the superstructure database as well as produce a "scenario difference file". Hence, even though you create multiple scenarios, **you only need to write to disk one database**.


In [ ]:
ndb.write_superstructure_db_to_brightway(name="my_db")

You can then analyze each scenario in Activity-Browser by loading the scenario difference file in the calculation setup window.


![Example superstructure](example_superDB.png)

### As a data package
Export a data package, which can be shared. Data packages can be read by [unfold](https://github.com/polca/unfold), and databases can be reproduced on other computers provided a local copy of ecoinvent is present. This way of sharing premise databases across users respects ecoinvent's EULA.

In [ ]:
ndb.write_datapackage()

### As a sparse matrix representation

Or export it as a sparse matrix representation.

This will export four files:

* "A_matrix.csv": matrix coordinates, values, and uncertainty columns for the technosphere
* "A_matrix_index.csv": labels for indices for A matrix of shape (name, reference product, unit, location, index)
* "B_matrix.csv": matrix coordinates, values, and uncertainty columns for the biosphere
* "B_matrix_index.csv": labels for indices for B matrix of shape (name, compartment, subcompartment, unit, index)

As a convenience, you can specify a directory where to store the exported matrices.
If the directory does not exist, it will be created.
If you leave it unspecified, they will be stored under `export/<model>/<pathway>/<year>` in your current working directory.


In [ ]:
ndb.write_db_to_matrices(filepath=r"C:/Users/sacchi_r/Downloads/exported_matrices")

The exported matrices have the following columns:

* "index of activity"
* "index of product"
* "value"
* "uncertainty type"
* "loc"
* "scale"
* "shape"
* "minimum"
* "maximum"
* "negative"
* "flip"

Where each row is an exchange (a technosphere exchange if looking at "A_matrix", else, a biosphere exchange). These matrices comply with the data structure expected by [bw_processing](https://pypi.org/project/bw-processing/) and can therefore be read directly by [bw2calc](https://pypi.org/project/bw2calc/) 2.0.x.


Otherwise, you can also do things manually. Here is an example of how to calculate GWP scores using the set of sparse matrices
export by `premise`.

In [ ]:
from scipy import sparse
#from pypardiso import spsolve <-- use pypardiso if you use an Intel chip, it's much faster!
from scipy.sparse.linalg import spsolve
from pathlib import Path
from csv import reader
import numpy as np

In [ ]:
# the directory to the set of files produced by premise
DIR = Path.cwd() / "export" / "remind" / "SSP2-PkBudg1000" / "2040"

# creates dict of activities <--> indices in A matrix
A_inds = dict()
with open(DIR / "A_matrix_index.csv", 'r') as read_obj:
    csv_reader = reader(read_obj, delimiter=";")
    for row in csv_reader:
        A_inds[(row[0], row[1], row[2], row[3])] = row[4]

A_inds_rev = {int(v):k for k, v in A_inds.items()}

# creates dict of bio flow <--> indices in B matrix
B_inds = dict()
with open(DIR / "B_matrix_index.csv", 'r') as read_obj:
    csv_reader = reader(read_obj, delimiter=";")
    for row in csv_reader:
        B_inds[(row[0], row[1], row[2], row[3])] = row[4]
        
B_inds_rev = {int(v):k for k, v in B_inds.items()}

# create a sparse A matrix
A_coords = np.genfromtxt(DIR / "A_matrix.csv", delimiter=";", skip_header=1)
I = A_coords[:, 0].astype(int)
J = A_coords[:, 1].astype(int)
A = sparse.csr_matrix((A_coords[:,2], (J, I)))

# create a sparse B matrix
B_coords = np.genfromtxt(DIR / "B_matrix.csv", delimiter=";", skip_header=1)
I = B_coords[:, 0].astype(int)
J = B_coords[:, 1].astype(int)
B = sparse.csr_matrix((B_coords[:,2] * -1, (I, J)), shape=(A.shape[0], len(B_inds)))

# a vector with a few GWP CFs
gwp = np.zeros(B.shape[1])
gwp[[int(B_inds[x]) for x in B_inds if x[0]=="Carbon dioxide, non-fossil, resource correction"]] = -1
#gwp[[int(B_inds[x]) for x in B_inds if x[0]=="Hydrogen"]] = 5
gwp[[int(B_inds[x]) for x in B_inds if x[0]=="Carbon dioxide, in air"]] = -1
gwp[[int(B_inds[x]) for x in B_inds if x[0]=="Carbon dioxide, non-fossil"]] = 1
gwp[[int(B_inds[x]) for x in B_inds if x[0]=="Carbon dioxide, fossil"]] = 1
gwp[[int(B_inds[x]) for x in B_inds if x[0]=="Carbon dioxide, from soil or biomass stock"]] = 1
gwp[[int(B_inds[x]) for x in B_inds if x[0]=="Carbon dioxide, to soil or biomass stock"]] = -1
gwp[[int(B_inds[x]) for x in B_inds if x[0]=="Carbon monoxide, fossil"]] = 4.06
gwp[[int(B_inds[x]) for x in B_inds if x[0]=="Methane, fossil"]] = 29.6

l_res = []
#for v in range(0, A.shape[0]):
# let's limit this to the first 3 activities of the matrix
for v in range(0, 3):
    f = np.float64(np.zeros(A.shape[0]))
    f[v] = 1
    A_inv = spsolve(A, f) # <-- this is too slow
    C = A_inv * B
    l_res.append((C * gwp).sum())


Print the results together with the name of the activity.

In [ ]:
[(k, v) for k, v in zip(l_res, list(A_inds_rev.values())[:10])]

# Incremental Database

You can use the class `IncrementalDatabase` to create a `brightway` database that lets you analyze the respective contribution of sectors in the environmental score of your LCA.

In [ ]:
ndb = IncrementalDatabase(
    scenarios=[
        {"model":"image", "pathway":"SSP2-M", "year":2040},
        {"model":"image", "pathway":"SSP2-M", "year":2050},
    ],
    source_db="ecoinvent-3.10-cutoff", # name of the database in the BW2 project. Must be a string.
    source_version="3.10", # 3.5-3.12 supported (consequential requires 3.8+)
    key="xxxxxxxxxxxxxxxxxxxxxxxxx",
    biosphere_name="ecoinvent-3.10-biosphere",
)


Then, you must choose the sectors to apply and the sequence of their implementation (and groupings if so desired).

In [ ]:
# in this case, we wish to apply transformations relating to 
# the electricity sector, the steel sector, as well as the 
# cement, cars and fuel sectors. However, in this example, 
# these last three will be considered altogether.

sectors = {
    "electricity": "electricity",
    "steel": "steel",
    "others": [
        "cement",
        "cars",
        "fuels"
    ]
}

ndb.update(sectors=sectors)

In [ ]:
# we export the database to brightway and open it in Activity Browser
# Just like a superstructure database, we load the scenario difference
# file premise has co-produced in the calculation setup window
# and we run an anlaysis. For more info, see how to conduct an analysis
# with a superstructure database.

ndb.write_increment_db_to_brightway(name="test increment", file_format="csv")

This lets us see the influence of each sector on the score of our LCA.


!["incremental_example"](incremental_example.png)

# Reports

## Scenario report

You can generate a spreadsheet report showing the main variables of the scenario you have selected to create your databases.
The report is saved in your working directory. Note that this report is generated automatically when exporting a database.

In [ ]:
ndb.generate_scenario_report()

## Changes report

You can generate a spreadsheet report of the changes made to the original database.
It gives an overview on:

* the datasets created
* the datasets modified
* some performance indicators
* scaling factors used to scale certain exchanges

There is also a "Validation" tab that shows any datasets which contains values or efficiencies that may seem incorrect.

The report is saved in your working directory. Note that this report is generated automatically when exporting a database.

In [ ]:
ndb.generate_change_report()